# Padlock Probe Designer

This notebook implements a pipeline to designe Padlock probes using the `oligo_designer_toolbox` package.

In [1]:
# import all the necassary pakages exept from the oligo_designer_toolsuite
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

import yaml
from Bio.SeqUtils import MeltingTemp as mt


## Define the parameters

The first thing to do is to define the parameters we want to use to generate the Padlock probes. 
A possible way to define all the parameters, that is flexible and reusable, is to use a configuration file. 
For this toutorial we will use the YAMl file `config/padlock_probe_designer.yaml`, you can have a look to understand which parameters are required and how the configuration file is structured.

Once the configuration file has been set up we have to read its content:

In [2]:
config_file = "../config/padlock_probe_designer.yaml"
with open(config_file, 'r') as y:
    config = yaml.safe_load(y)
dir_output = os.path.join(os.path.dirname(os.getcwd()), config["dir_output"]) # create the complete path for teh output directory

## Oligo sequences generation

Now we can start to actually build the pipeline, we will start by generating all the possible probes with length between the maximum and minimum value given belonging to the genes defined in the config file. The probes will be saved in a nested dicionary with the following structure: 

[gene][probe_id][probe_feature].


For this we need a `CustomDB`, or a class that inherits form it (e.g. `NCBIDB` and `EnsemblDB`) and call the method `create_oligos_DB`. These classes differ on how the the fasta and the gtf filesused to compute the sequences are obtained. The first one oses local files while the others dowload them form the NCBI or Ensambl ftp server.

In [3]:
from oligo_designer_toolsuite.IO import CustomDB, NcbiDB, EnsemblDB

# define the database class
if config["source"] == "ncbi":
    # dowload the fasta files formthe NCBI server
    database = NcbiDB(
        probe_length_min=config["probe_length_min"],
        probe_length_max=config["probe_length_max"],
        species=config["species"],
        annotation_release=config["annotation_release"],
        n_jobs=config["n_jobs"],
        dir_output=dir_output,
        min_probes_per_gene=config["min_probes_per_gene"],
        )
elif config["source"] == "ensembl":
    # dowload the fasta files formthe NCBI server
    database = EnsemblDB(
        probe_length_min=config["probe_length_min"],
        probe_length_max=config["probe_length_max"],
        species=config["species"],
        annotation_release=config["annotation_release"],
        n_jobs=config["n_jobs"],
        dir_output=dir_output,
        min_probes_per_gene=config["min_probes_per_gene"],
        )
elif config["source"] == "custom":
    # use already dowloaded files
    database = CustomDB(
        probe_length_min=config["probe_length_min"],
        probe_length_max=config["probe_length_max"],
        species=config["species"],
        genome_assembly=config["genome_assembly"],
        annotation_release=config["annotation_release"],
        annotation_source=config["annotation_source"],
        file_annotation=config["file_annotation"],
        file_sequence=config["file_sequence"],
        n_jobs=config["n_jobs"],
        dir_output=dir_output,
        min_probes_per_gene=config["min_probes_per_gene"],
        )

# read the genes file
with open(config["file_genes"]) as handle:
    lines = handle.readlines()
    genes = [line.rstrip() for line in lines]
    
#generate the sequences
database.create_oligos_DB(genes=genes)

### Dictionary structure

Here is an example of how the nested dictionary is structured. We print a very simple dictionary with 1 probe.

In [4]:
gene = "AGRN"
probe_id = "AGRN_1"

sample_oligos_DB = {}
sample_oligos_DB[gene] = {}
sample_oligos_DB[gene][probe_id] = database.oligos_DB[gene][probe_id]
print(sample_oligos_DB)

{'AGRN': {'AGRN_1': {'probe_sequence': Seq('AGTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCG'), 'transcript_id': ['NM_001305275.2:XM_005244749.4:XM_011541429.3:NM_198576.4:XM_047419836.1'], 'exon_id': ['NM_001305275.2_exon1:XM_005244749.4_exon1:XM_011541429.3_exon1:NM_198576.4_exon1:XM_047419836.1_exon1'], 'chromosome': '1', 'start': [1020119], 'end': [1020157], 'strand': '+', 'length': 38}}}


### Read and write

These classes deal with everything that is related with the management of the dataset. In particular, beyond creatig the dataset, they can also read and write the oligo sequences in a **tsv** or **gtf** fromat. The methods `read_oligos_DB` and `write_oligos_DB` have exactly this purpose.

Therefore, it is possible to save the current state of the dictionary during the pipeline and to retrive form a previous stage if an error uccurred.

In [5]:
if config["write_intermediate_steps"]:
    database.write_oligos_DB(format=config["file_format"], dir_oligos_DB="oligos_creation")

## Property filters

Once all the possible sequences are created, we apply a first filtering process based on the sequences properties (e.g. melting temperature or GC content). This is useful to reduce the amount of sequences we have to deal with in the next stages and discard all the sequences that are not suited for the experiment scope.

Each property filter is a calss that inherits from the Abstact Base Class `PreFilterBase` They have a method called `apply` that takes the `oligos_DB` and returns it filtered. To make this process smooth and modular the class `PropertyFilter` allows to apply several filters one after the other. It takes in input a list of filter classes and applies them sequentailly to the `oligos_DB` returning the final filterd version of the database. Additionally, all the necessary sequence features computed by the filters are stored in the `oligos_DB` for possible later use.

To create new property filters follow the Abstact Base Class requirements in `PreFilterBase`.

In [6]:
from oligo_designer_toolsuite.oligo_property_filter import (
    PropertyFilter,
    MaskedSequences,
    GCContent, 
    MeltingTemperature, 
    PadlockArms
)

# the melting temperature params need to be preprocessed
Tm_params = config["Tm_parameters"]["shared"].copy()
Tm_params.update(config["Tm_parameters"]["property_filter"])
Tm_params["nn_table"] = getattr(mt, Tm_params["nn_table"])
Tm_params["tmm_table"] = getattr(mt, Tm_params["tmm_table"])
Tm_params["imm_table"] = getattr(mt, Tm_params["imm_table"])
Tm_params["de_table"] = getattr(mt, Tm_params["de_table"])

Tm_correction_param = config["Tm_correction_parameters"]["shared"].copy()
Tm_correction_param.update(config["Tm_correction_parameters"]["property_filter"])

# initialize the filters clasees
masked_sequences = MaskedSequences()
gc_content = GCContent(GC_content_min=config["GC_content_min"], GC_content_max=config["GC_content_max"])
melting_temperature = MeltingTemperature(
    Tm_min=config["Tm_min"], 
    Tm_max=config["Tm_max"], 
    Tm_parameters=Tm_params, 
    Tm_correction_parameters=Tm_correction_param
)
padlock_arms = PadlockArms(
    min_arm_length=config["min_arm_length"],
    max_arm_Tm_dif=config["max_arm_Tm_dif"],
    arm_Tm_min=config["arm_Tm_min"],
    arm_Tm_max=config["arm_Tm_max"],
    Tm_parameters=Tm_params,
    Tm_correction_parameters=Tm_correction_param,
)
# create the list of filters
filters = [masked_sequences, gc_content, melting_temperature, padlock_arms]

# initialize the property filter class
property_filter = PropertyFilter(filters=filters, write_genes_with_insufficient_probes=config["write_removed_genes"])
# filter the database
database = property_filter.apply(database=database, n_jobs=config["n_jobs"])
# write the intermediate result in a file
if config["write_intermediate_steps"]:
    database.write_oligos_DB(format=config["file_format"], dir_oligos_DB="property_filter")

## Specificity filters

Generally, in experiments using DNA sequences one of the main problems that can occur are off-target binding of the oligo sequences designed. To avoid this one can decide to delete all the sequences that also match regions of the DNA outside the gene they belong to.

The classes in the subpackage `oligo_speificity_filters` detect these probes using aligne methods such as Blast and Bowtie and remove them from the database. The currently implemeted classes are: `ExactMatches`, `Blastn`, `Bowtie`, `Bowtie2`, `BowtieSeedRegion`, look at the documentation to understand more in detail their features. As before a second class `SpecificityFilter` takes a list of all the filters we want to apply, and applies them sequentially to the `oligos_DB`. 

For our pipeline, we will use `ExactMatches`, `Blastn`, `BowtieSeedRegion`, where for the latter the seed region will be generated with `LigationRegionCreation` (look at the documentation to understand what seed region means).

However, alignement methods need a reference fasta file to detect the off-target regions. The `Custom_DB` provides also the possibility to generate this reference region with the method `create_reference_DB`.

**Remark:** in future versions the database classes will be split, one will deal only with the oligo sequences, while the other only the the reference.

In [ ]:
from oligo_designer_toolsuite.oligo_specificity_filter import (
    SpecificityFilter,
    ExactMatches,
    LigationRegionCreation,
    BowtieSeedRegion,
    Blastn,
)

dir_specificity = os.path.join(dir_output, "specificity_temporary") # folder where the temporary files will be written

# generate the reference
database.create_reference_DB() # use standard parameters 

# intialize the filter classes
exact_mathces = ExactMatches(dir_specificity=dir_specificity)
seed_ligation = LigationRegionCreation(ligation_region_size=config["ligation_region_size"])
seed_region = BowtieSeedRegion(dir_specificity=dir_specificity, seed_region_creation=seed_ligation)
blastn = Blastn(
    dir_specificity=dir_specificity, 
    word_size=config["word_size"],
    percent_identity=config["percent_identity"],
    coverage=config["coverage"],
)
filters = [exact_mathces, seed_region, blastn]

# initialize the specificity filter class
specificity_filter = SpecificityFilter(filters=filters, write_genes_with_insufficient_probes=config["write_removed_genes"])
# filte r the database
database = specificity_filter.apply(database=database, n_jobs=config["n_jobs"])
# write the intermediate result
if config["write_intermediate_steps"]:
    database.write_oligos_DB(format=config["file_format"], dir_oligos_DB="specificity_filter")

## Probeset generation

In the next step of the pipeline the probes will be choosen according to their theoretical efficiency in the experiment scope (e.g. how well they bind to the target in the DNA). Each probe will receive a score computed by a class that inherits from `ProbeScoringBase`. Later, the sequences will be organized in sets and a class inheriting from `SetScoringBase` will give a general efficiency score to the set. At the end the best sets will be selected and sored.

It is required that the each array of sequences  contains probes that do not overlap. In fact, if two probes were overlapping, they would compete to bind to the same section of DNA and their efficiency would drop significantly. Therefore, it is extremely important to consider only sets of non-overlapping sequences.

The class `ProbesetGenerator` takes the scoring strategies and tries to find, among all the feasible non-overlapping sets of probes, the sets with the best efficiency scores. These sets will be save in a pandas DataFrame with the following structure:


 probeset_id | probe_0  | probe_1  | probe_2  |  ...  | probe_n  | set_score_1 | set_score_2 |  ...  
------------ | -------- | -------- | -------- | ----- | -------- | ----------- | ----------- | ------:
 0           | AGRN_184 | AGRN_133 | AGRN_832 |  ...  | AGRN_706 | 0.3445      | 1.2332      |  ...  


In [8]:
from oligo_designer_toolsuite.oligo_efficiency import(
    PadlockProbeScoring,
    PadlockSetScoring,
)
from oligo_designer_toolsuite.oligo_selection import ProbesetGenerator, padlock_heuristic_selection

# initialize the scoring classes
probes_scoring = PadlockProbeScoring(
    Tm_min=config["Tm_min"],
    Tm_opt=config["Tm_opt"],
    Tm_max=config["Tm_max"],
    GC_content_min=config["GC_content_min"],
    GC_content_opt=config["GC_content_opt"],
    GC_content_max=config["GC_content_max"],
    Tm_weight=config["Tm_weight"],
    GC_weight=config["GC_weight"],
)
set_scoring = PadlockSetScoring()

# initialize the probeset generator class
probeset_generator = ProbesetGenerator(
    probeset_size=config["probeset_size"], 
    min_probeset_size=config["min_probeset_size"],
    probes_scoring=probes_scoring,
    set_scoring=set_scoring,
    heurustic_selection=padlock_heuristic_selection,
    write_genes_with_insufficient_probes=config["write_removed_genes"]
)

# generate the probeset
database = probeset_generator.apply(database=database, n_sets=config["n_sets"], n_jobs=config["n_jobs"])
# write the intermediate result
if config["write_intermediate_steps"]:
    database.write_probesets(dir_probesets="probesets")

## Last step

Once the best probesets are generated each experiment design might require (or not) an addtional step. In the case of the Padlock probe designer the last step consists in designing the final padlock probe sequences.

In [9]:
from oligo_designer_toolsuite.experiment_specific import PadlockSequenceDesigner

# preprocessing of themelting temperature parameters
Tm_params = config["Tm_parameters"]["shared"].copy()
Tm_params.update(config["Tm_parameters"]["detection_oligo"])
Tm_params["nn_table"] = getattr(mt, Tm_params["nn_table"])
Tm_params["tmm_table"] = getattr(mt, Tm_params["tmm_table"])
Tm_params["imm_table"] = getattr(mt, Tm_params["imm_table"])
Tm_params["de_table"] = getattr(mt, Tm_params["de_table"])

Tm_correction_param = config["Tm_correction_parameters"]["shared"].copy()
Tm_correction_param.update(config["Tm_correction_parameters"]["detection_oligo"])

# initilize the padlock sequence designer class
padlock_sequence_designer = PadlockSequenceDesigner(
    detect_oligo_length_min=config["detect_oligo_length_min"],
    detect_oligo_length_max=config["detect_oligo_length_max"],
    detect_oligo_Tm_opt=config["detect_oligo_Tm_opt"],
    Tm_parameters=Tm_params,
    Tm_correction_parameters=Tm_correction_param
)
# generate the padlock sequence
padlock_sequence_designer.design_padlocks(database=database)